In [ ]:
import cv2
import numpy as np
import os
import time

class DNN_FaceDetector:
    def __init__(self, model_path, config_path, confidence_threshold=0.4):
        self.net = cv2.dnn.readNetFromCaffe(config_path, model_path)
        self.threshold = confidence_threshold
        self.trackers = []
        self.tracking = False
        self.last_detection_time = 0
        self.detection_interval = 2

    def detect_faces(self, frame):
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 117.0, 123.0))
        self.net.setInput(blob)
        detections = self.net.forward()
        
        faces = []
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > self.threshold:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                faces.append((x1, y1, x2 - x1, y2 - y1))
        
        return faces

    def process_frame(self, frame):
        current_time = time.time()
        
        if not self.tracking or len(self.trackers) == 0 or (current_time - self.last_detection_time > self.detection_interval):
            faces = self.detect_faces(frame)
            self.trackers = []
            
            for bbox in faces:
                if hasattr(cv2, 'legacy'):
                    tracker = cv2.legacy.TrackerKCF_create()
                else:
                    tracker = cv2.TrackerKCF.create()
                tracker.init(frame, bbox)
                self.trackers.append((tracker, bbox))
            
            self.tracking = True
            self.last_detection_time = current_time
            return [(True, bbox) for bbox in faces]
        
        else:
            results = []
            new_trackers = []
            lost_trackers = 0
            
            for tracker, bbox in self.trackers:
                success, new_bbox = tracker.update(frame)
                if success:
                    results.append((True, new_bbox))
                    new_trackers.append((tracker, new_bbox))
                else:
                    results.append((False, None))
                    lost_trackers += 1
            
            self.trackers = new_trackers
            
            if lost_trackers == len(results) or (len(results) == 1 and lost_trackers == 1):
                self.tracking = False
                return self.process_frame(frame)
            
            return results

def process_all_videos(input_folder, output_subfolder):
    output_folder = os.path.join(input_folder, output_subfolder)
    os.makedirs(output_folder, exist_ok=True)
    
    detector = DNN_FaceDetector(
        "res10_300x300_ssd_iter_140000_fp16.caffemodel",
        "deploy.prototxt"
    )
    
    video_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    
    if not video_files:
        print(f"No video files found in {input_folder}")
        return
    
    total_start_time = time.time()
    
    for video_file in video_files:
        input_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, f"processed_{video_file}")
        
        print(f"\nProcessing {video_file}...")
        start_time = time.time()
        
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"Error opening video file {input_path}")
            continue
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        last_update_time = start_time
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            results = detector.process_frame(frame)
            
            for success, bbox in results:
                if success:
                    x, y, w, h = [int(v) for v in bbox]
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, "Face", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            out.write(frame)
            
            current_time = time.time()
            if current_time - last_update_time >= 1.0:
                elapsed_time = current_time - start_time
                progress = frame_count / total_frames
                if progress > 0:
                    remaining_time = (elapsed_time / progress) - elapsed_time
                else:
                    remaining_time = 0
                
                elapsed_str = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
                remaining_str = time.strftime("%H:%M:%S", time.gmtime(remaining_time))
                
                print(f"Elapsed: {elapsed_str} | Remaining: {remaining_str} | Progress: {frame_count}/{total_frames} frames | Faces: {len(results)}", end='\r')
                last_update_time = current_time
        
        cap.release()
        out.release()
        print(f"\nFinished processing {video_file}. Saved to {output_path}")
        print(f"Processing time: {time.time() - start_time:.2f} seconds")
    
    print(f"\nAll videos processed. Total processing time: {time.time() - total_start_time:.2f} seconds")

def main():
    data_folder = "../Data"
    processed_subfolder = "Processed_Data"
    
    process_all_videos(data_folder, processed_subfolder)

if __name__ == "__main__":
    main()


Processing test1.mp4...
Elapsed: 00:00:04 | Remaining: 00:00:00 | Progress: 267/279 frames | Faces: 1
Finished processing test1.mp4. Saved to ../Data\Processed_Data\processed_test1.mp4
Processing time: 4.19 seconds

Processing test2.mp4...
Elapsed: 00:00:13 | Remaining: 00:00:01 | Progress: 578/647 frames | Faces: 0
Finished processing test2.mp4. Saved to ../Data\Processed_Data\processed_test2.mp4
Processing time: 13.91 seconds

Processing test3.mp4...
Elapsed: 00:00:04 | Remaining: 00:00:00 | Progress: 316/359 frames | Faces: 1
Finished processing test3.mp4. Saved to ../Data\Processed_Data\processed_test3.mp4
Processing time: 4.44 seconds

Processing test4.mp4...
Elapsed: 00:00:02 | Remaining: 00:00:00 | Progress: 208/276 frames | Faces: 1
Finished processing test4.mp4. Saved to ../Data\Processed_Data\processed_test4.mp4
Processing time: 2.71 seconds

All videos processed. Total processing time: 25.24 seconds
